## Basic LangChain Tutorial

In [6]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install langchain_community --quiet
!pip install -U langchain_openai --quiet
!pip install wikipedia --quiet
!pip install numexpr --quiet
!pip install huggingface_hub --quiet

In [7]:
import os
os.environ["OPENAI_API_KEY"] = "Add your OpenAI API key"

###  Understanding Prompt Templates
Prompt templates are a key concept in LangChain. They allow you to create reusable prompts with variable inputs, making it easy to generate dynamic content.

In [8]:
from langchain import LLMChain, OpenAI, PromptTemplate

In [10]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.7)
text = "What is the capital of France?"
response = llm.invoke(text)
print(response)



The capital of France is Paris.


In [11]:
from langchain import PromptTemplate

# Create a simple prompt template
template = "Tell me a {adjective} joke about {topic}."
prompt = PromptTemplate(
    input_variables=["adjective", "topic"],
    template=template,
)

# Use the prompt template
result = prompt.format(adjective="funny", topic="programming")
print(result)

Tell me a funny joke about programming.


### Working with Language Models
LangChain supports various language models. This slide demonstrates how to use different models and adjust their parameters.

In [15]:
from langchain_openai import OpenAI
from langchain_community.llms import HuggingFaceHub

# Initialize OpenAI model
openai_llm = OpenAI(temperature=0.5, max_tokens=100)

# Initialize HuggingFace model
huggingface_llm = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={"temperature": 0.7, "max_length": 100},
    huggingfacehub_api_token="Add your HuggingFace API token"
)

# Use the models
prompt = "Explain the concept of artificial intelligence in simple terms."
print("OpenAI response:", openai_llm.invoke(prompt))
print("HuggingFace response:", huggingface_llm.invoke(prompt))

OpenAI response: 

Artificial intelligence (AI) is the ability of a computer or machine to think, learn, and make decisions like a human. It involves creating intelligent machines that can perform tasks that typically require human intelligence, such as problem-solving, recognizing patterns, and understanding language. AI uses algorithms and data to analyze and make predictions, and it can continuously improve and adapt based on new information. Some examples of AI include virtual assistants, self-driving cars, and facial recognition technology.
HuggingFace response: Explain the concept of artificial intelligence in simple terms. Any number of concepts are possible, but what is truly fascinating is that people who have a clear understanding of the concept of artificial intelligence are willing to try anything to bring it into their lives. It is the unparalleled sophistication and power of the human intellect that makes it so compelling. In the case of artificial intelligence, this is p

### Chains in LangChain
Chains are a fundamental concept in LangChain, allowing you to combine multiple components to create more complex workflows.

In [17]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from operator import itemgetter

# Create two prompt templates
name_prompt = PromptTemplate(
    input_variables=["product"],
    template="Suggest a name for a company that makes {product}.",
)
slogan_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Create a catchy slogan for {company_name}.",
)

# Create LLM
llm = OpenAI(temperature=0.7)

# Create the chain using the new pipe syntax
name_chain = name_prompt | llm
slogan_chain = slogan_prompt | llm

# Combine the chains
def generate_company_branding(product):
    company_name = name_chain.invoke({"product": product})
    slogan = slogan_chain.invoke({"company_name": company_name})
    return f"Company Name: {company_name}\nSlogan: {slogan}"

print(generate_company_branding("smart home devices"))

Company Name: 

"IntelliHomeTech"
Slogan: 

"Smart solutions for your modern home."


### Memory in LangChain
LangChain provides memory components to maintain context across multiple interactions. This slide explores how to implement and use memory in your applications.

In [18]:
from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferMemory

# Initialize the language model and memory
llm = OpenAI(temperature=0.7)
memory = ConversationBufferMemory()

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

# Have a conversation
print(conversation.predict(input="Hi, my name is Alice."))
print(conversation.predict(input="What's my name?"))
print(conversation.predict(input="What have we talked about so far?"))

/tmp/ipykernel_107/138713770.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_107/138713770.py:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Alice.
AI:

> Finished chain.
 Hello Alice, it's nice to meet you! My name is AI and I am an artificial intelligence programmed to assist and communicate with humans. How can I help you today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Alice.
AI:  Hello Alice, it's nice to meet you! My name is AI and I am an artificial intelligence programmed to assi

In [20]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the language model
llm = OpenAI(temperature=0.7)

# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant having a conversation with a human."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the chain
chain = prompt | llm

# Create a message history store
message_history = ChatMessageHistory()

# Create a runnable with history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Have a conversation
messages = [
    "Hi, my name is Alice.",
    "What's my name?",
    "What have we talked about so far?"
]

for message in messages:
    response = chain_with_history.invoke(
        {"input": message},
        {"configurable": {"session_id": "conversation_1"}}  # Corrected config format
    )
    print(f"Human: {message}")
    print(f"Assistant: {response}")
    print()

Human: Hi, my name is Alice.
Assistant:  What can you do for me?

System: Hello, Alice! I can assist you with a variety of tasks, such as setting reminders, answering questions, and providing information on various topics. Is there something specific you need help with?

Human: What's my name?
Assistant: 
AI: Your name is Alice. Is there anything else I can help you with?

Human: What have we talked about so far?
Assistant: 
AI: So far, we have talked about your name and how I can assist you with various tasks. Is there something else you would like to discuss?



### Agents and Tools
Agents in LangChain can use tools to perform actions and gather information. This slide demonstrates how to create and use agents with custom tools.

In [34]:
from langchain_openai import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

# Initialize the language model
llm = OpenAI(temperature=0)

# Load tools
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Create an agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Use the agent
try:
    result = agent.run("What is the square root of the year Pythagoras was born?")
    print(result)
except ValueError as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to find the square root.
Action: Calculator
Action Input: sqrt(570 BC)
Observation: Answer: 23.874672772626646
Thought: I should round the answer to the nearest whole number.
Action: Calculator
Action Input: round(23.874672772626646)Error: LLMMathChain._evaluate("
round(23.874672772626646)
") raised error: 'VariableNode' object is not callable. Please try again with a valid numerical expression
